In [33]:
 def cloudscore(image):
        '''
        Inner function for computing cloud score such that we can remove 
        bad images from the landsat collections we download.
        Implementation in javascript can be found of Google Earth Engine 
        website under (landsat algorithms), translation to python by KH.
        Further help from Nicholas Clinton at 
        https://gis.stackexchange.com/questions/252685/filter-landsat-images-base-on-cloud-cover-over-a-region-of-interest
        '''
        cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')
        cloudiness = cloud.reduceRegion(ee.Reducer.mean(),
                                        geometry=region,
                                        scale=30)
        image = image.set(cloudiness)
        return image

### Plan
1. Create a an alps df and a himalayas df
2. Find the top n largest glaciers and if there are several outlines pick the most recent src_date (date of the glacier at the time of the image used in the outline)
3. Download the images from earth engine with all bands

In [34]:
import ee
import os
import geemap
import geopandas as gpd
from shapely import wkt
import numpy as np
from datetime import datetime
from datetime import timedelta
import pandas as pd
import matplotlib.pyplot as plt
import json
from qgrid import show_grid
from collections import Counter

### 1. Create a df

In [41]:
out_dir = os.path.join(os.path.expanduser("~"), "Desktop","projects","glacier_segmentation","glims_data")
file_name = "master_glims_df.csv"
df = pd.read_csv(os.path.join(out_dir,file_name), index_col = 0) #read in data

In [42]:
df.shape

(548914, 8)

In [43]:
df['date'] = [datetime.strptime(dat.split("T")[0], '%Y-%m-%d') for dat in list(df.src_date)] #convert string dates to datetime
df = df.loc[df['date'] > datetime(2005,1,1)] #select glaciers with polygons drawn after 2005

In [44]:
df = df[~df['geog_area'].isin(['Svalbard, Jan Mayen, and Bouvet', "Various (GlobGlacier)", 'Randolph Glacier Inventory; Umbrella RC for merging the RGI into GLIMS','Antarctic Peninsula','Various (NSIDC)'])]

In [45]:
df = df[(df.db_area > 1) & (df.db_area < 50)]

In [46]:
#create df which is a df of unique glaciers sorted by area, descending
df = df.reset_index(drop = True)
clean_copy = df.copy()
s = set()
for i in range(df.shape[0]):
    print(i)
    if df.iloc[i,0] in s:
        clean_copy = clean_copy.drop(i, axis = 0)
        continue
    else:
        s.add(df.iloc[i,0])
df = clean_copy

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150


3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912


5500
5501
5502
5503
5504
5505
5506
5507
5508
5509
5510
5511
5512
5513
5514
5515
5516
5517
5518
5519
5520
5521
5522
5523
5524
5525
5526
5527
5528
5529
5530
5531
5532
5533
5534
5535
5536
5537
5538
5539
5540
5541
5542
5543
5544
5545
5546
5547
5548
5549
5550
5551
5552
5553
5554
5555
5556
5557
5558
5559
5560
5561
5562
5563
5564
5565
5566
5567
5568
5569
5570
5571
5572
5573
5574
5575
5576
5577
5578
5579
5580
5581
5582
5583
5584
5585
5586
5587
5588
5589
5590
5591
5592
5593
5594
5595
5596
5597
5598
5599
5600
5601
5602
5603
5604
5605
5606
5607
5608
5609
5610
5611
5612
5613
5614
5615
5616
5617
5618
5619
5620
5621
5622
5623
5624
5625
5626
5627
5628
5629
5630
5631
5632
5633
5634
5635
5636
5637
5638
5639
5640
5641
5642
5643
5644
5645
5646
5647
5648
5649
5650
5651
5652
5653
5654
5655
5656
5657
5658
5659
5660
5661
5662
5663
5664
5665
5666
5667
5668
5669
5670
5671
5672
5673
5674
5675
5676
5677
5678
5679
5680
5681
5682
5683
5684
5685
5686
5687
5688
5689
5690
5691
5692
5693
5694
5695
5696
5697
5698
5699


7299
7300
7301
7302
7303
7304
7305
7306
7307
7308
7309
7310
7311
7312
7313
7314
7315
7316
7317
7318
7319
7320
7321
7322
7323
7324
7325
7326
7327
7328
7329
7330
7331
7332
7333
7334
7335
7336
7337
7338
7339
7340
7341
7342
7343
7344
7345
7346
7347
7348
7349
7350
7351
7352
7353
7354
7355
7356
7357
7358
7359
7360
7361
7362
7363
7364
7365
7366
7367
7368
7369
7370
7371
7372
7373
7374
7375
7376
7377
7378
7379
7380
7381
7382
7383
7384
7385
7386
7387
7388
7389
7390
7391
7392
7393
7394
7395
7396
7397
7398
7399
7400
7401
7402
7403
7404
7405
7406
7407
7408
7409
7410
7411
7412
7413
7414
7415
7416
7417
7418
7419
7420
7421
7422
7423
7424
7425
7426
7427
7428
7429
7430
7431
7432
7433
7434
7435
7436
7437
7438
7439
7440
7441
7442
7443
7444
7445
7446
7447
7448
7449
7450
7451
7452
7453
7454
7455
7456
7457
7458
7459
7460
7461
7462
7463
7464
7465
7466
7467
7468
7469
7470
7471
7472
7473
7474
7475
7476
7477
7478
7479
7480
7481
7482
7483
7484
7485
7486
7487
7488
7489
7490
7491
7492
7493
7494
7495
7496
7497
7498


...

13650
13651
13652
13653
13654
13655
13656
13657
13658
13659
13660
13661
13662
13663
13664
13665
13666
13667
13668
13669
13670
13671
13672
13673
13674
13675
13676
13677
13678
13679
13680
13681
13682
13683
13684
13685
13686
13687
13688
13689
13690
13691
13692
13693
13694
13695
13696
13697
13698
13699
13700
13701
13702
13703
13704
13705
13706
13707
13708
13709
13710
13711
13712
13713
13714
13715
13716
13717
13718
13719
13720
13721
13722
13723
13724
13725
13726
13727
13728
13729
13730
13731
13732
13733
13734
13735
13736
13737
13738
13739
13740
13741
13742
13743
13744
13745
13746
13747
13748
13749
13750
13751
13752
13753
13754
13755
13756
13757
13758
13759
13760
13761
13762
13763
13764
13765
13766
13767
13768
13769
13770
13771
13772
13773
13774
13775
13776
13777
13778
13779
13780
13781
13782
13783
13784
13785
13786
13787
13788
13789
13790
13791
13792
13793
13794
13795
13796
13797
13798
13799
13800
13801
13802
13803
13804
13805
13806
13807
13808
13809
13810
13811
13812
13813
13814
13815
1381

15190
15191
15192
15193
15194
15195
15196
15197
15198
15199
15200
15201
15202
15203
15204
15205
15206
15207
15208
15209
15210
15211
15212
15213
15214
15215
15216
15217
15218
15219
15220
15221
15222
15223
15224
15225
15226
15227
15228
15229
15230
15231
15232
15233
15234
15235
15236
15237
15238
15239
15240
15241
15242
15243
15244
15245
15246
15247
15248
15249
15250
15251
15252
15253
15254
15255
15256
15257
15258
15259
15260
15261
15262
15263
15264
15265
15266
15267
15268
15269
15270
15271
15272
15273
15274
15275
15276
15277
15278
15279
15280
15281
15282
15283
15284
15285
15286
15287
15288
15289
15290
15291
15292
15293
15294
15295
15296
15297
15298
15299
15300
15301
15302
15303
15304
15305
15306
15307
15308
15309
15310
15311
15312
15313
15314
15315
15316
15317
15318
15319
15320
15321
15322
15323
15324
15325
15326
15327
15328
15329
15330
15331
15332
15333
15334
15335
15336
15337
15338
15339
15340
15341
15342
15343
15344
15345
15346
15347
15348
15349
15350
15351
15352
15353
15354
15355
1535

16572
16573
16574
16575
16576
16577
16578
16579
16580
16581
16582
16583
16584
16585
16586
16587
16588
16589
16590
16591
16592
16593
16594
16595
16596
16597
16598
16599
16600
16601
16602
16603
16604
16605
16606
16607
16608
16609
16610
16611
16612
16613
16614
16615
16616
16617
16618
16619
16620
16621
16622
16623
16624
16625
16626
16627
16628
16629
16630
16631
16632
16633
16634
16635
16636
16637
16638
16639
16640
16641
16642
16643
16644
16645
16646
16647
16648
16649
16650
16651
16652
16653
16654
16655
16656
16657
16658
16659
16660
16661
16662
16663
16664
16665
16666
16667
16668
16669
16670
16671
16672
16673
16674
16675
16676
16677
16678
16679
16680
16681
16682
16683
16684
16685
16686
16687
16688
16689
16690
16691
16692
16693
16694
16695
16696
16697
16698
16699
16700
16701
16702
16703
16704
16705
16706
16707
16708
16709
16710
16711
16712
16713
16714
16715
16716
16717
16718
16719
16720
16721
16722
16723
16724
16725
16726
16727
16728
16729
16730
16731
16732
16733
16734
16735
16736
16737
1673

18131
18132
18133
18134
18135
18136
18137
18138
18139
18140
18141
18142
18143
18144
18145
18146
18147
18148
18149
18150
18151
18152
18153
18154
18155
18156
18157
18158
18159
18160
18161
18162
18163
18164
18165
18166
18167
18168
18169
18170
18171
18172
18173
18174
18175
18176
18177
18178
18179
18180
18181
18182
18183
18184
18185
18186
18187
18188
18189
18190
18191
18192
18193
18194
18195
18196
18197
18198
18199
18200
18201
18202
18203
18204
18205
18206
18207
18208
18209
18210
18211
18212
18213
18214
18215
18216
18217
18218
18219
18220
18221
18222
18223
18224
18225
18226
18227
18228
18229
18230
18231
18232
18233
18234
18235
18236
18237
18238
18239
18240
18241
18242
18243
18244
18245
18246
18247
18248
18249
18250
18251
18252
18253
18254
18255
18256
18257
18258
18259
18260
18261
18262
18263
18264
18265
18266
18267
18268
18269
18270
18271
18272
18273
18274
18275
18276
18277
18278
18279
18280
18281
18282
18283
18284
18285
18286
18287
18288
18289
18290
18291
18292
18293
18294
18295
18296
1829

19639
19640
19641
19642
19643
19644
19645
19646
19647
19648
19649
19650
19651
19652
19653
19654
19655
19656
19657
19658
19659
19660
19661
19662
19663
19664
19665
19666
19667
19668
19669
19670
19671
19672
19673
19674
19675
19676
19677
19678
19679
19680
19681
19682
19683
19684
19685
19686
19687
19688
19689
19690
19691
19692
19693
19694
19695
19696
19697
19698
19699
19700
19701
19702
19703
19704
19705
19706
19707
19708
19709
19710
19711
19712
19713
19714
19715
19716
19717
19718
19719
19720
19721
19722
19723
19724
19725
19726
19727
19728
19729
19730
19731
19732
19733
19734
19735
19736
19737
19738
19739
19740
19741
19742
19743
19744
19745
19746
19747
19748
19749
19750
19751
19752
19753
19754
19755
19756
19757
19758
19759
19760
19761
19762
19763
19764
19765
19766
19767
19768
19769
19770
19771
19772
19773
19774
19775
19776
19777
19778
19779
19780
19781
19782
19783
19784
19785
19786
19787
19788
19789
19790
19791
19792
19793
19794
19795
19796
19797
19798
19799
19800
19801
19802
19803
19804
1980

In [12]:
show_grid(df.head(2000), grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

### 2. Find the top k largest glaciers

In [11]:
ids = list(Counter(df.glac_id).keys()) #get list of unique ids

In [12]:
len(ids)

17090

In [16]:
df_sub = df.sample(n=3000) #get top 3000 largest glaciers

### 3. Pull the images from EE with all the bounds

In [18]:
# Trigger the authentication flow
ee.Authenticate()

# Initialize the library
ee.Initialize()

Enter verification code: 4/1AX4XfWgzUouTG_fseFs2eWTH4yKE2ZHcCU-j5TB027U3j6QDwxn9miVD-94

Successfully saved authorization token.


In [47]:
df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
gdf = gpd.GeoDataFrame(df, geometry = "geometry")

In [48]:
gdf.iloc[:,:-1].to_file("polygons/polys_sub.shp")

In [28]:
df_sub_2 = df_sub.iloc[1488:,:]

In [30]:
#everything will go in this loop once I know it works
cloud_tol=15
delta = timedelta(days = 200)
out_dir = os.path.join(os.path.expanduser('~'),"Desktop","projects","glacier_segmentation", "training")

for idx, glims_id in enumerate(df_sub_2.glac_id):
    one_glacier = df_sub.loc[df_sub.glac_id == glims_id]
    poly_date = list(one_glacier.date)[0]
    begDate = poly_date-delta
    endDate = poly_date+delta

    bbox = one_glacier.bboxes
    bbox = eval(list(bbox)[0])
    region = ee.Geometry.Polygon(list(bbox))
    
    if poly_date > datetime(2013,1,1):
        print('2013')
        col = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
        col = col.filterDate(begDate,endDate)
        col = col.filterBounds(region)
        withCloudiness = col.map(algorithm=cloudscore)
        colSelected = withCloudiness.filter(ee.Filter.lt('cloud', cloud_tol))
        colSelected = colSelected.select(['B2','B3','B4','B5','B6','B7'])
        colList = colSelected.toList(col.size())
        colSize = colSelected.size().getInfo()
    
    #elif poly_date > datetime(1999,1,1):
        #print('1999')
        #col = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA')
        #col = col.filterDate(begDate,endDate)
        #col = col.filterBounds(region)
        #withCloudiness = col.map(algorithm=cloudscore)
        #colSelected = withCloudiness.filter(ee.Filter.lt('cloud', cloud_tol))
        #colSelected = colSelected.select(['B1','B2','B3','B4','B5','B7'])
        #colList = colSelected.toList(col.size())
        #colSize = colSelected.size().getInfo()
    
    else:
        print('old')
        col = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA')
        col = col.filterDate(begDate,endDate)
        col = col.filterBounds(region)
        withCloudiness = col.map(algorithm=cloudscore)
        colSelected = withCloudiness.filter(ee.Filter.lt('cloud', cloud_tol))
        colSelected = colSelected.select(['B1','B2','B3','B4','B5','B7'])
        colList = colSelected.toList(col.size())
        colSize = colSelected.size().getInfo()
        
    
    print("Number of Images: ",colSize)
    
    if colSize == 0:
        continue
    
    filename = "{}.tif".format(glims_id) 
    image = ee.Image(colList.get(0))
    geemap.ee_export_image(image, filename = os.path.join(out_dir, filename), scale = 30, region = region, file_per_band = False)
                           

old
Number of Images:  3
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G091573E33425N.tif
old
Number of Images:  0
old
Number of Images:  0
old
Number of Images:  6
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G086768E31441N.tif
old
Number of Images:  4
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G096553E29109N.tif
old
Number of Images:  3
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G096502E29430N.tif
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G082663E29891N.tif
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G076684E3

Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G090855E28106N.tif
old
Number of Images:  6
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G097296E28902N.tif
old
Number of Images:  0
old
Number of Images:  0
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G089405E30079N.tif
old
Number of Images:  0
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G075282E37067N.tif
old
Number of Images:  3
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G089954E28218N.tif
old
Number of Images:  3
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G082545E29236N.tif
old
Number of Images:  0


Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G084292E43493N.tif
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G079820E34401N.tif
old
Number of Images:  0
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G078184E32662N.tif
old
Number of Images:  3
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G079958E30233N.tif
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G073768E39123N.tif
old
Number of Images:  3
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G243389E50246N.tif
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded

Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G084998E43524N.tif
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G075158E37278N.tif
old
Number of Images:  3
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G076606E36559N.tif
old
Number of Images:  10
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G077697E31731N.tif
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G286971E49269S.tif
old
Number of Images:  0
old
Number of Images:  0
old
Number of Images:  0
old
Number of Images:  10
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G287983E42419S.tif
old
Number of Images:  

Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G042439E43234N.tif
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G094356E43057N.tif
old
Number of Images:  5
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G290249E30455S.tif
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G008230E46588N.tif
old
Number of Images:  0
old
Number of Images:  3
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G077727E35584N.tif
old
Number of Images:  0
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G080681E41960N.tif
old
Number of Images:  3
Generating URL ...
Please

...

Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G081129E42128N.tif
old
Number of Images:  0
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G289893E31312S.tif
old
Number of Images:  4
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G010681E46458N.tif
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G088396E27976N.tif
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G232238E53439N.tif
old
Number of Images:  6
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G286764E50376S.tif
old
Number of Images:  0
old
Number of Images:  6
Generating URL ...
Please

Number of Images:  4
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G096978E29048N.tif
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G240555E53386N.tif
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G082151E42773N.tif
old
Number of Images:  9
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G242247E52441N.tif
old
Number of Images:  0
old
Number of Images:  5
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G093057E29664N.tif
old
Number of Images:  0
old
Number of Images:  16
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G086820E3134

Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G224494E59114N.tif
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G076452E36729N.tif
old
Number of Images:  0
old
Number of Images:  0
old
Number of Images:  0
old
Number of Images:  8
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G078402E34648N.tif
old
Number of Images:  3
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G092282E27884N.tif
old
Number of Images:  0
old
Number of Images:  0
old
Number of Images:  5
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G223654E59326N.tif
old
Number of Images:  0
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects

Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G009167E46880N.tif
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G095670E29783N.tif
old
Number of Images:  3
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G232729E56040N.tif
old
Number of Images:  1
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G096423E29430N.tif
old
Number of Images:  9
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G228961E56619N.tif
old
Number of Images:  0
old
Number of Images:  0
old
Number of Images:  0
old
Number of Images:  0
old
Number of Images:  0
old
Number of Images:  4
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\tra

Number of Images:  0
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G286936E52937S.tif
old
Number of Images:  4
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G007577E62248N.tif
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G282486E09155S.tif
old
Number of Images:  14
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G080613E30328N.tif
old
Number of Images:  2
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G224160E59190N.tif
old
Number of Images:  7
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\training\G008861E46827N.tif
old
Number of Imag

#### Playing around with geemap

In [ ]:
one_glacier

In [ ]:
cloud_tol=10
delta = timedelta(days = 50)
out_dir = os.path.join(os.path.expanduser('~'),"Desktop","projects","glacier_segmentation", "training")

one_glacier = df_alps.loc[df_alps.glac_id == 'G007026E45991N']
poly_date = list(one_glacier.date)[0]
begDate = poly_date-delta
endDate = poly_date+delta

bbox = one_glacier.bboxes
bbox = eval(list(bbox)[0])
region = ee.Geometry.Polygon(list(bbox))

colL8 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA')
colL8 = col.filterDate(begDate,endDate)
colL8 = col.filterBounds(region)

withCloudiness = colL8.map(algorithm=cloudscore)
colSelectedL8 = withCloudiness.filter(ee.Filter.lt('cloud', cloud_tol))
colSelectedL8 = colSelectedL8.select(['B1','B2','B3','B4','B5','B7'])
colListL8 = colSelectedL8.toList(colL8.size())
colSizeL8 = colSelectedL8.size().getInfo()

print("Number of Images: ",colSizeL8)

out_dir = os.path.join(os.path.expanduser('~'),"Desktop","projects","glacier_segmentation")
filename = "landsat.tif" 
image = ee.Image(colListL8.get(0))
geemap.ee_export_image(image, filename = os.path.join(out_dir, filename), scale = 90, region = region, file_per_band = False)
trient = geemap.ee_to_numpy(image, region = region, default_value = 0)

In [ ]:
plt.imshow(trient[:,:,3:7])